# Hidden shift algorithm

## From Filipa Peres (INL)


[Bravyi and Gosset](https://arxiv.org/abs/1601.07601v3) and [Bravyi *et. al*](https://arxiv.org/abs/1808.00128v2) state that the hidden shift problem is particularly suited to benchmark their quantum circuit simulator because:

1. the output is deterministic;
and
2. the $T$ count of the algorithm is easily controlled by choosing a suitable bent function.

Following the description presented in the Appendix F of [[Bravyi and Gosset]](https://arxiv.org/abs/1601.07601v3), I will implement the hidden shift algorithm here as well.

---



In [2]:
import qiskit
import random
import os

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram

In [3]:
from HSA import HSA_gen

In [44]:
# number of qubits
n = 9

n_ccz = 1 # implies t_count = 14 * n_ccz
g = 2 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

In [45]:
# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)


The "hidden" string is:  011010111


#### <span style=color:green>Circuit generation!</span>
    

In [46]:
qc = HSA_gen (n, n_ccz, g, s, Toffoli= False, measure=False, remove_pairs_H=True)


CircuitError: 'Number of items in qubits parameter (5) does not match number of qubits in the circuit (4).'

#### <span style=color:green>Circuit statistics:</span>

In [40]:

print('Depth: ',qc.depth())
print ('Width (qubits): ', qc.num_qubits)
g_list =  qc.count_ops()
print ('Gates: ', g_list)
num_g = 0
for key,value in g_list.items():
    num_g += value
print ('Total Gates: ', num_g)
    

Depth:  43
Width (qubits):  26
Gates:  OrderedDict([('h', 56), ('cx', 44), ('s', 38), ('t', 14)])
Total Gates:  152


In [41]:
#qc.draw ('mpl')

#### <span style=color:green>Circuit execution!</span>
    

In [42]:
backend = Aer.get_backend('statevector_simulator')

# Launch the job
job_sim = backend.run(qc, shots=1)
    
# Grab the results from the job.
result_sim = job_sim.result()

y = result_sim.get_statevector(qc, decimals=8)
#print(y)
  

In [43]:
  
print()
# Prepare a my_statevector with all states sorted and real part only
my_statevector = {}
sv_dict = y.to_dict()
#print (sv_dict)
format_str = '{{0:0{0}b}}'.format(n)
for i in range(2**n):
    bin_str = format_str.format(i)
    if bin_str in sv_dict:
        my_statevector[bin_str] = sv_dict[bin_str]
    else:
        my_statevector[bin_str] = 0.
for key in my_statevector:
    if abs(my_statevector[key])>1.e-5:
        print ('{0} : {1:.8f}'.format(key,my_statevector[key]))


00011000010001101011000000 : 1.00000000-0.00000000j


In [ ]:
print ('That\'s all, folks')